In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
import importlib
from astropy.cosmology import FlatLambdaCDM

from utils_orbs.orb_paths import SetupPaths
import utils_orbs.readsubfHDF5Py3 as readSub
# from utils.read_group_cats import ReadCats
from utils_orbs.merger_trees import TraceMergerTree
from utils_orbs.readMergerHistoryFiles import FindHistory
from utils_orbs.vectorCorrection import vectorCorrection as vector

paths = SetupPaths()

In [ ]:
plt.show();
plt.rcParams.update({'font.size':20,"xtick.direction":"in","ytick.direction":"in", 
                     "xtick.top":True, "ytick.right":True,"text.usetex":False,
                     "xtick.labelsize":18,"ytick.labelsize":18})

## 

In [ ]:
# check if there are any common ids between snapshot 40 and 41
orbit_dict = {}
for snap in [40,41,42]:
    f = h5py.File(f"{paths.path_data}hail-mary/orbitdata_{snap}.hdf5",'r')
    orbit_dict[str(snap)] = {}
    for key, val in f.items():
        orbit_dict[str(snap)][key] = np.array(val)
    f.close()

f = h5py.File(f"{paths.path_data}snapshot_data.hdf5",'r')
snap_dict = {}
for key, val in f.items():
    snap_dict[key] = np.array(val)
f.close()

In [ ]:
xx = orbit_dict['40']['PairKey']
yy = orbit_dict['41']['PairKey']
zz = orbit_dict['42']['PairKey']

In [ ]:
print(len(xx),len(yy),len(zz))

In [ ]:
alpha = np.concatenate((xx,yy,zz))
print(len(alpha))
print(len(np.unique(alpha)))

## Calculate the number of total unique pairs in the catalog

In [11]:
f = h5py.File(f"{paths.path_data}hail-mary/orbitdata_1.hdf5",'r')
f.keys()

<KeysViewHDF5 ['GroupFlag', 'GroupNum', 'GroupRvir', 'InfallRedshift', 'InfallSnapshot', 'MergeFlag', 'MergeRedshift', 'MergeSnapshot', 'PairKey', 'Redshift', 'RelativeVelocity', 'Scale', 'Separations', 'SeparationsComoving', 'SeparationsScaled', 'Snapshot', 'StellarMass1', 'StellarMass2', 'StellarMassRatio', 'SubfindID1', 'SubfindID2', 'SubhaloMass1', 'SubhaloMass2', 'SubhaloPos1', 'SubhaloPos2', 'SubhaloVel1', 'SubhaloVel2']>

In [12]:
pairkeys = np.array([])
for snap in range(100)[1:]:
    if snap == 48:
        continue
    
    f = h5py.File(f"{paths.path_data}hail-mary/orbitdata_{snap}.hdf5",'r')
    pairkeys = np.concatenate((pairkeys,f['PairKey']))
#     pairkeys.append(list(f['PairKey']))
    f.close()

In [13]:
print("Total pairs",len(pairkeys))
print("Unique pairs",len(np.unique(pairkeys)))

Total pairs 71429
Unique pairs 22213


## Test composite dictionary w snap=40

In [ ]:
empty_frame = {}
for i in orbit_dict['40'].keys():
    empty_frame[i] = []
    
    
orbit = {}
f = h5py.File(f"{paths.path_data}hail-mary/orbitdata_40.hdf5",'r')
for key, val in f.items():
    orbit[key] = np.array(val)
f.close()

for hua in range(len(orbit['PairKey'])):
    print(hua)
    pk = orbit['PairKey'][hua]
    if pk in empty_frame['PairKey']:
        continue
    else:
        for key in empty_frame.keys():
            if key in ["Redshift","Scale","Snapshot"]:
                continue
            empty_frame[key].append(orbit[key][hua])


In [ ]:
empty_frame.keys()

In [ ]:
for key,val in empty_frame.items():
    print(key, np.shape(val))

## Compile ALL orbits

In [17]:
all_frame = {}
for i in orbit_dict['40'].keys():
    all_frame[i] = []
    
    
for snapnum in range(100):    
    print(snapnum)

    if snapnum == 48:
        continue
    
    orbit = {}
    f = h5py.File(f"{paths.path_data}hail-mary/orbitdata_{snapnum}.hdf5",'r')
    for key, val in f.items():
        orbit[key] = np.array(val)
    f.close()

    for hua in range(len(orbit['PairKey'])):
        pk = orbit['PairKey'][hua]
        
        for key in all_frame.keys():
            if key not in ["Redshift","Scale","Snapshot"]:
                all_frame[key].append(orbit[key][hua])
            elif key=="Snapshot":
                all_frame[key].append(snapnum)

                


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


### Save full orbit catalog

In [18]:
info_dict = {"Redshift":"Redshift of snapshot",
                      "Scale":"Scale of snapshot",
                      "Snapshot":"Snapshot number",
                      "GroupNum":"FoF group number at snapshot",
                      "SubfindID1":"Subhalo ID of primary at selected redshift",
                      "SubfindID2":"Subhalo ID of secondary at selected redshift",
                      "SubhaloMass1":"Subhalo mass at selected redshift in 1e10*Msun",
                      "SubhaloMass2":"Subhalo mass at selected redshift in 1e10*Msun",
                      "StellarMass1":"Stellar mass from median AM at selected redshift",
                      "StellarMass2":"Stellar mass from median AM at selected redshift",
                      "StellarMassRatio":"Stellar mass ratio at selected redshift",
                      "MergeFlag":"True if subhalos will merge", 
                      "MergeRedshift":"Redshift of 'merger'",
                      "MergeSnapshot":"Snapshot at which 'merger' has occured",
                      "InfallRedshift":"First redshift where Group is the same",
                      "InfallSnapshot":"First snapshot where Group is the same",
                      "PairKey":"Unique identifying key for each pair (same between snapshots for same pair)",
                      "GroupFlag":"True if subhalos are in the same group",
                      "GroupRvir":"Radius of primary group in kpc",
                      "Separations":"Physical separation between pair in kpc",
                      "SeparationsComoving":"Comoving separation between pair in ckpc/h",
                      "SeparationsScaled":"Separation scaled by radius of primary group, dimensionless",
                      "RelativeVelocity":"Relative velocity between pair in km/s",
                      "SubhaloPos1":"Position of subhalo in ckpc/h",
                      "SubhaloPos2":"Position of subhalo in ckpc/h",
                      "SubhaloVel1":"Velocity of subhalo in km/s",
                      "SubhaloVel2":"Velocity of subhalo in km/s"
                    }


f = h5py.File(f"../../data/hail-mary/all_orbits.hdf5", 'w')
header_dict = {"Details":"Full sample of low mass major pair orbits (including duplicates)",
        "Simulation":"TNG100-1"}

dset = f.create_group('/Header')
for key in header_dict.keys():
    dset.attrs[key] = header_dict[key]

for key, val in all_frame.items():
    if key not in ["Redshift","Scale"]:
        val = np.array(val)
        dset = f.create_dataset(f'/{key}', 
                                shape=val.shape,
                                dtype=val.dtype)
        dset.attrs[key] = info_dict[key]
        dset[:] = val

f.close()

## Compile all unique orbits

In [19]:
unique_frame = {}
for i in orbit_dict['40'].keys():
    unique_frame[i] = []
    
    
for snapnum in range(100):    
    print(snapnum)
    if snapnum == 48:
        continue
    
    orbit = {}
    f = h5py.File(f"{paths.path_data}hail-mary/orbitdata_{snapnum}.hdf5",'r')
    for key, val in f.items():
        orbit[key] = np.array(val)
    f.close()

    for hua in range(len(orbit['PairKey'])):
        pk = orbit['PairKey'][hua]
        if pk in unique_frame['PairKey']:
            continue
        else:
            for key in unique_frame.keys():
                if key not in ["Redshift","Scale","Snapshot"]:
                    unique_frame[key].append(orbit[key][hua])
                elif key=="Snapshot":
                    unique_frame[key].append(snapnum)

                


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


### Save unique orbits

In [22]:
info_dict = {"Redshift":"Redshift of snapshot",
                      "Scale":"Scale of snapshot",
                      "Snapshot":"Snapshot number",
                      "GroupNum":"FoF group number at snapshot",
                      "SubfindID1":"Subhalo ID of primary at selected redshift",
                      "SubfindID2":"Subhalo ID of secondary at selected redshift",
                      "SubhaloMass1":"Subhalo mass at selected redshift in 1e10*Msun",
                      "SubhaloMass2":"Subhalo mass at selected redshift in 1e10*Msun",
                      "StellarMass1":"Stellar mass from median AM at selected redshift",
                      "StellarMass2":"Stellar mass from median AM at selected redshift",
                      "StellarMassRatio":"Stellar mass ratio at selected redshift",
                      "MergeFlag":"True if subhalos will merge", 
                      "MergeRedshift":"Redshift of 'merger'",
                      "MergeSnapshot":"Snapshot at which 'merger' has occured",
                      "InfallRedshift":"First redshift where Group is the same",
                      "InfallSnapshot":"First snapshot where Group is the same",
                      "PairKey":"Unique identifying key for each pair (same between snapshots for same pair)",
                      "GroupFlag":"True if subhalos are in the same group",
                      "GroupRvir":"Radius of primary group in kpc",
                      "Separations":"Physical separation between pair in kpc",
                      "SeparationsComoving":"Comoving separation between pair in ckpc/h",
                      "SeparationsScaled":"Separation scaled by radius of primary group, dimensionless",
                      "RelativeVelocity":"Relative velocity between pair in km/s",
                      "SubhaloPos1":"Position of subhalo in ckpc/h",
                      "SubhaloPos2":"Position of subhalo in ckpc/h",
                      "SubhaloVel1":"Velocity of subhalo in km/s",
                      "SubhaloVel2":"Velocity of subhalo in km/s"
                    }


f = h5py.File(f"../../data/hail-mary/unique_orbits.hdf5", 'w')
header_dict = {"Details":"Full UNIQUE sample of low mass major pair orbits",
        "Simulation":"TNG100-1"}

dset = f.create_group('/Header')
for key in header_dict.keys():
    dset.attrs[key] = header_dict[key]

for key, val in unique_frame.items():
    if key not in ["Redshift","Scale"]:
        val = np.array(val)
        dset = f.create_dataset(f'/{key}', 
                                shape=val.shape,
                                dtype=val.dtype)
        dset.attrs[key] = info_dict[key]
        dset[:] = val

f.close()